<a href="https://colab.research.google.com/github/jonrtaylor/twitch/blob/master/example_model_kazutsugi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

train_datalink = 'https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz'

df_train = pd.read_csv(train_datalink)

In [ ]:
from xgboost import XGBRegressor

model = XGBRegressor(max_depth = 5, learning_rate = 0.01, n_estimators = 2000, n_jobs = -1, colsample_bytree = 0.1)#, tree_method = 'gpu_hist')

y = df_train.target.values

features = [c for c in df_train if c.startswith("feature")]

X = df_train[features].values

model.fit(X, y)

[20:24:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.1, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=2000,
             n_jobs=-1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
del df_train
del X
del y

def correlation_score(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0,1]

TOURNAMENT_DATAPATH = 'https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz'
df_tournament = pd.read_csv(TOURNAMENT_DATAPATH)
df_val = df_tournament[df_tournament.data_type == "validation"].copy()
y_val = df_val.target.values
X_val = df_val[features].values
preds = model.predict(X_val)

In [ ]:
nomi_score = correlation_score(y_val, preds)
print(nomi_score)

del df_val
del y_val
del X_val
del preds

In [ ]:
predictions = model.predict(df_tournament[features].values)
predictions_df = df_tournament["id"].to_frame()
predictions_df["prediction"] = predictions
predictions_df.to_csv('example_predictions_twitch.csv', index=False)

In [ ]:
from google.colab import files
files.download("example_predictions_twitch.csv")